In [13]:
import glob, os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from itertools import combinations
from sklearn.manifold import MDS

In [14]:
src_pattern = r'/workspace/Krrish/Silent_Speech/dataset_sony/Normalized_dataset/recordings/*/*.csv'
features = ["theta", "x", "y"]
C = len(features)
print("1. Loading & Normalizing PER SUBJECT...")

files = glob.glob(src_pattern)

X, words, subjects = [], [], []
T = 150

for fp in files:
    try:
        df = pd.read_csv(fp)
        data = df[features].values
        #normalization
        scaler = StandardScaler()
        data = scaler.fit_transform(data)
        
        data = data[300:]   # remove calibration

        if len(data) < 1500:
            print("Warning: shorter than expected", fp)
        
        segments = []
        for i in range(10):
            start = i * 150
            end = start + 150
            segments.append(data[start:end])
        
        segments = np.stack(segments)

        if n_chunks == 0:
            continue

        data = data[:n_chunks * T].reshape(n_chunks, T, len(features))
        X.append(data)

        word = os.path.basename(os.path.dirname(fp))
        subj = os.path.basename(fp).replace(".csv", "")

        words.extend([word] * n_chunks)
        subjects.extend([subj] * n_chunks)

    except Exception as e:
        print("Skipping:", fp, e)

X = np.vstack(X)  # (N, T, 5)
print(X.shape)
print(list(set(words)))
print(list(set(subjects)))

1. Loading & Normalizing PER SUBJECT...
(2177, 150, 3)
['Dhadkan', 'Chah', 'Peshab', 'Dawai', 'Hospital', 'Bahaar', 'Paani', 'Peed', 'Khangh', 'Bukhar', 'Doctor', 'Dard', 'Jukham', 'Saah', 'Ulti', 'Bhuk', 'Neend', 'Khoon', 'Piyaas', 'Kabz', 'Ghabrahat', 'Kamjori', 'Gharde', 'Sardard', 'Chakkar']
['madhav', 'KamalPreet', 'Surindar', 'Anupam', 'Bansbir', 'Armman', 'anubhavjot', 'Amish', 'harsh', 'Jaskaran']


In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(words)

num_classes = len(le.classes_)
print("Number of classes:", num_classes)


Number of classes: 25


In [16]:
import numpy as np

unique_subjects = list(set(subjects))
print("Subjects:", unique_subjects)

test_subject = unique_subjects[-1]  # pick one

train_idx = [i for i, s in enumerate(subjects) if s != test_subject]
test_idx  = [i for i, s in enumerate(subjects) if s == test_subject]

X_train = X[train_idx]
y_train = y[train_idx]

X_test = X[test_idx]
y_test = y[test_idx]

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Subjects: ['madhav', 'KamalPreet', 'Surindar', 'Anupam', 'Bansbir', 'Armman', 'anubhavjot', 'Amish', 'harsh', 'Jaskaran']
Train shape: (1947, 150, 3)
Test shape: (230, 150, 3)


In [17]:
X_train = np.transpose(X_train, (0, 2, 1))
X_test  = np.transpose(X_test,  (0, 2, 1))

print("After transpose:", X_train.shape)


After transpose: (1947, 3, 150)


In [18]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)

X_test_t  = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_t  = torch.tensor(y_test, dtype=torch.long).to(device)


In [19]:
class JawCNN(nn.Module):
    def __init__(self, num_classes):
        super(JawCNN, self).__init__()

        self.conv1 = nn.Conv1d(3, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(64)

        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x).squeeze(-1)
        return self.fc(x)


In [20]:
model = JawCNN(num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


In [21]:
epochs = 40
batch_size = 64

for epoch in range(epochs):
    model.train()
    perm = torch.randperm(X_train_t.size(0))
    total_loss = 0

    for i in range(0, X_train_t.size(0), batch_size):
        idx = perm[i:i+batch_size]
        xb = X_train_t[idx]
        yb = y_train_t[idx]

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss {total_loss:.4f}")


Epoch 0, Loss 100.1847
Epoch 5, Loss 96.3180
Epoch 10, Loss 94.2479
Epoch 15, Loss 92.7842
Epoch 20, Loss 91.2477
Epoch 25, Loss 89.8471
Epoch 30, Loss 88.9557
Epoch 35, Loss 87.6607


In [22]:
model.eval()

with torch.no_grad():
    logits = model(X_test_t)
    probs = torch.softmax(logits, dim=1)

    # Top-1
    top1 = torch.argmax(probs, dim=1)
    top1_acc = (top1 == y_test_t).float().mean().item()

    # Top-2
    top2 = torch.topk(probs, k=2, dim=1).indices
    correct_top2 = 0
    for i in range(len(y_test_t)):
        if y_test_t[i] in top2[i]:
            correct_top2 += 1
    top2_acc = correct_top2 / len(y_test_t)

print("Top-1:", top1_acc)
print("Top-2:", top2_acc)


Top-1: 0.08695651590824127
Top-2: 0.10869565217391304


In [23]:
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(
    y_test_t.cpu().numpy(),
    top1.cpu().numpy()
)

print(cm)


[[0 0 0 0 0 2 0 0 1 0 0 6 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0 0 4 0 1 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 3 0 0 4 0 0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 1 0 0 2 0 0 0 1 0 4 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 5 0 0 0 0 0 0 0 0 1 2 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 4 0 1 0 0 0 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 8 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 9 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 3 0 0 0 0 1 0 0 0 1 0 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 6 0 1 0 0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 3 0 0 0 2 2 1 0 0 1 0 0 0 0]
 [1 0 0 6 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 3 0 2 0 0 0 1 0 0 1 2 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0 0 2 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 2]
 [0 0 0 0 0 0 1 0 3 0 0 2 0 0 0 1 1 2 0 0 0 0 

In [24]:
model.eval()

with torch.no_grad():
    logits = model(X_test_t)
    probs = torch.softmax(logits, dim=1)

    # Top-1
    top1 = torch.argmax(probs, dim=1)
    top1_acc = (top1 == y_test_t).float().mean().item()

    # Top-2
    top2 = torch.topk(probs, k=2, dim=1).indices
    correct_top2 = sum(
        [1 for i in range(len(y_test_t)) if y_test_t[i] in top2[i]]
    )
    top2_acc = correct_top2 / len(y_test_t)

print("Top-1:", top1_acc)
print("Top-2:", top2_acc)


Top-1: 0.08695651590824127
Top-2: 0.10869565217391304


In [25]:
from collections import Counter
print(Counter(words))


Counter({'Dawai': 100, 'Dard': 100, 'Khoon': 100, 'Piyaas': 90, 'Doctor': 90, 'Hospital': 90, 'Peed': 90, 'Bukhar': 90, 'Paani': 90, 'Ulti': 90, 'Jukham': 89, 'Neend': 89, 'Kabz': 89, 'Chah': 89, 'Sardard': 89, 'Bhuk': 86, 'Kamjori': 80, 'Ghabrahat': 80, 'Bahaar': 80, 'Gharde': 80, 'Chakkar': 80, 'Peshab': 79, 'Saah': 79, 'Khangh': 79, 'Dhadkan': 79})


In [26]:
from collections import Counter

print("Train:", Counter(y_train))
print("Test:", Counter(y_test))


Train: Counter({np.int64(6): 90, np.int64(5): 90, np.int64(16): 90, np.int64(21): 80, np.int64(8): 80, np.int64(11): 80, np.int64(19): 80, np.int64(2): 80, np.int64(18): 80, np.int64(24): 80, np.int64(12): 79, np.int64(17): 79, np.int64(22): 79, np.int64(15): 79, np.int64(13): 79, np.int64(3): 79, np.int64(23): 79, np.int64(1): 76, np.int64(14): 70, np.int64(9): 70, np.int64(0): 70, np.int64(10): 70, np.int64(4): 70, np.int64(20): 69, np.int64(7): 69})
Test: Counter({np.int64(21): 10, np.int64(8): 10, np.int64(14): 10, np.int64(6): 10, np.int64(1): 10, np.int64(12): 10, np.int64(5): 10, np.int64(11): 10, np.int64(17): 10, np.int64(19): 10, np.int64(20): 10, np.int64(9): 10, np.int64(16): 10, np.int64(2): 10, np.int64(13): 10, np.int64(7): 10, np.int64(18): 10, np.int64(0): 10, np.int64(3): 10, np.int64(24): 10, np.int64(10): 10, np.int64(23): 10, np.int64(4): 10})


In [27]:
from scipy.signal import resample

T = 150
data_resampled = resample(data, T, axis=0)
X.append(data_resampled[np.newaxis, :, :])


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [28]:
for fp in files[:5]:   # check a few first
    df = pd.read_csv(fp)
    total_len = len(df)
    print(fp, total_len)


/workspace/Krrish/Silent_Speech/dataset_sony/Normalized_dataset/recordings/Piyaas/Amish.csv 1500
/workspace/Krrish/Silent_Speech/dataset_sony/Normalized_dataset/recordings/Piyaas/Armman.csv 1500
/workspace/Krrish/Silent_Speech/dataset_sony/Normalized_dataset/recordings/Piyaas/Bansbir.csv 1500
/workspace/Krrish/Silent_Speech/dataset_sony/Normalized_dataset/recordings/Piyaas/Surindar.csv 1500
/workspace/Krrish/Silent_Speech/dataset_sony/Normalized_dataset/recordings/Piyaas/KamalPreet.csv 1500
